In [22]:
import pandas as pd
import numpy as np
import os

def filepath(f):
    d = os.path.join(os.getcwd(), 'data', 'fraud_detection_data', f)
    return d    

# Importing testing datasets
test_inpatient = pd.read_csv(filepath("Test_Inpatientdata.csv"))
test_outpatient = pd.read_csv(filepath("Test_Outpatientdata.csv"))
test_beneficiary = pd.read_csv(filepath("Test_Beneficiarydata.csv"))
test_label = pd.read_csv(filepath("Test.csv"))

In [23]:
##Replacing 2 with 0 for chronic conditions ,that means chroniv condition No is 0 and yes is 1
test_beneficiary = test_beneficiary.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2, 
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2, 
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

test_beneficiary = test_beneficiary.replace({'RenalDiseaseIndicator': 'Y'}, 1)

In [24]:
#Combining test data
test_inpatient['inpatient'] = 1
test_outpatient['inpatient'] = 0
#test = pd.concat([test_inpatient, test_outpatient], axis=0, sort=False)
test=pd.merge(test_outpatient,test_inpatient,
                              left_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode','inpatient'],
                              right_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode','inpatient']
                              ,how='outer')

test_alldata=pd.merge(test,test_beneficiary,left_on='BeneID',right_on='BeneID',how='inner')


C:\Users\aengu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\reshape\merge.py:1213: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


In [25]:
# Drop constant columns
for col in test_alldata.columns:
    if test_alldata[col].nunique() <= 1:
        test_alldata.drop([col], inplace=True, axis=1)
        print(col, "dropped as it is a constant.")

ClmProcedureCode_6 dropped as it is a constant.


In [26]:
#Merging with Provider
test_allprovider=pd.merge(test_label,test_alldata,on='Provider')

In [27]:
#Removing unecessary columns
drop_cols = [
    'NoOfMonths_PartACov', 
    'NoOfMonths_PartBCov', 
    'State', 
    'County', 
    'AttendingPhysician',
    'OperatingPhysician', 
    'OtherPhysician', 
    'DiagnosisGroupCode',
    "RenalDiseaseIndicator"
    ]
test_allprovider=test_allprovider.drop(drop_cols, axis=1)

In [28]:
#Read in icd9 data
diagnosis=pd.read_csv("data/medical_code_data/icd9_diagnosis.csv", dtype={'code': str})
procedures=pd.read_csv("data/medical_code_data/icd9_procedure.csv", dtype={'code': str})

In [29]:
diagnosis_dict = {}
unique_diagnosis = diagnosis['description_1'].unique()
for (i,row) in diagnosis.iterrows():
    diagnosis_dict[row["code"]] = row["description_1"]

procedures_dict = {}
unique_procedures = procedures['description_1'].unique()
for (i,row) in procedures.iterrows():
    procedures_dict[row["code"]] = row["description_1"]


In [30]:
unique_procedures

array(['Procedures And Interventions , Not Elsewhere Classified',
       'Operations On The Nervous System',
       'Operations On The Endocrine System', 'Operations On The Eye',
       'Other Miscellaneous Diagnostic And Therapeutic Procedures',
       'Operations On The Ear',
       'Operations On The Nose, Mouth, And Pharynx',
       'Operations On The Respiratory System',
       'Operations On The Cardiovascular System',
       'Operations On The Hemic And Lymphatic System',
       'Operations On The Digestive System',
       'Operations On The Urinary System',
       'Operations On The Male Genital Organs',
       'Operations On The Female Genital Organs',
       'Obstetrical Procedures',
       'Operations On The Musculoskeletal System',
       'Operations On The Integumentary System',
       'Miscellaneous Diagnostic And Therapeutic Procedures'],
      dtype=object)

In [31]:
diagnoses_code_in_table = {}
for i,d in enumerate(unique_diagnosis):
    diagnoses_code_in_table[d] = i+1

procedures_code_in_table = {}
for i,p in enumerate(unique_procedures):
    procedures_code_in_table[p] = i+1



In [32]:
# Create a list of columns to process
columns_to_process = ['ClmDiagnosisCode_' + str(i) for i in range(1, 11)]
not_present_set_diagnosis = set()

# Replace the codes with the descriptions
diagnosis_descriptor = []
for (i,row) in test_allprovider.iterrows():
    descriptors = set()
    for col in columns_to_process:
        adm = False
        try:
            diag_num = str(diagnosis_dict[row[col]])
            new_code2 = diagnoses_code_in_table[diag_num]
        except:
            if pd.isnull(row[col]):
                continue
            else:
                set.add(not_present_set_diagnosis, row[col])
                continue
        descriptors.add(new_code2)
    diagnosis_descriptor.append(descriptors)


In [33]:
adm_diag = []
for (i,row) in test_allprovider.iterrows():
    ind = False
    value = row["ClmAdmitDiagnosisCode"]
    for col in columns_to_process:
        if ind == True:
            continue
        if pd.isnull(value):
            continue
        if str(value) == str(row[col]):
            adm_diag.append(1)
            ind = True
    if ind == False:
        adm_diag.append(0)
len(adm_diag)

test_allprovider["admitDiagInFinalDiagnosis"] = adm_diag

In [34]:
columns_to_process2 = ['ClmProcedureCode_' + str(i) for i in range(1, 6)]

not_present_set_procedure = set()
procedures_descriptor = []
for (i,row) in test_allprovider.iterrows():
    procedures = set()
    for col in columns_to_process2:
        try:
            n = str(int(row[col]))
            new_code = procedures_dict[n]
            new_code2 = procedures_code_in_table[new_code]
        except:
            if pd.isnull(row[col]):
                continue
            else:
                set.add(not_present_set_procedure, str(int(row[col])))
                continue
        procedures.add(new_code2)
    procedures_descriptor.append(procedures)

In [35]:
not_present_set_procedure

{'309'}

In [36]:
diagnoses_columns = {}

for j in [i for i in diagnoses_code_in_table.values()]:
    diagnoses_columns[j] = []

for d in diagnosis_descriptor:
    for k,v in diagnoses_columns.items():
        if k in d:
            v.append(1)
        else:
            v.append(0)

procedures_columns = {}

for j in [i for i in procedures_code_in_table.values()]:
    procedures_columns[j] = []

for p in procedures_descriptor:
    for k,v in procedures_columns.items():
        if k in p:
            v.append(1)
        else:
            v.append(0)
    

In [37]:
final_diagnosis_cols = {}
for k, name in zip(diagnoses_columns.keys(), diagnoses_code_in_table.values()):
    new_name = "diagnosis_"+str(name)
    final_diagnosis_cols[new_name] = diagnoses_columns[k]

final_procedures_cols = {}
for k, name in zip(procedures_columns.keys(), procedures_code_in_table.values()):
    new_name = "procedure_"+str(name)
    final_procedures_cols[new_name] = procedures_columns[k]


In [38]:
test_allprovider['diagnosis_descriptor'] = diagnosis_descriptor
test_allprovider['procedures_descriptor'] = procedures_descriptor
test_allprovider.drop(
    columns_to_process + columns_to_process2,
    axis = 1,
    inplace = True
)

test_allprovider.drop(
    ["ClmAdmitDiagnosisCode"],
    axis = 1,
    inplace = True
)

test_allprovider.head()

,Provider,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,DeductibleAmtPaid,inpatient,AdmissionDt,DischargeDt,...,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,admitDiagInFinalDiagnosis,diagnosis_descriptor,procedures_descriptor
0,PRV51002,BENE13342,CLM129901,2009-01-10,2009-01-10,100,0.0,0,NaN,NaN,...,0,0,0,0,0,1310,590,0,"{8, 16, 19}",{}
1,PRV51002,BENE13915,CLM163936,2009-01-29,2009-01-29,40,0.0,0,NaN,NaN,...,0,0,0,22000,3204,1020,910,0,"{16, 19, 5}",{}
2,PRV51002,BENE13915,CLM348696,2009-05-09,2009-05-09,70,0.0,0,NaN,NaN,...,0,0,0,22000,3204,1020,910,0,{19},{}
3,PRV51002,BENE13915,CLM500906,2009-08-01,2009-08-01,100,0.0,0,NaN,NaN,...,0,0,0,22000,3204,1020,910,0,{7},{}
4,PRV51002,BENE14575,CLM272820,2009-03-29,2009-03-29,400,0.0,0,NaN,NaN,...,1,0,1,8000,13136,1520,730,0,{2},{}


In [39]:
final_test_df = pd.concat(
    [
        pd.concat(
            [
                test_allprovider,
                pd.DataFrame(
                    final_diagnosis_cols
                )
            ],
            axis = 1
        ),
        pd.DataFrame(
            final_procedures_cols
        )
    ], 
    axis = 1
)

In [43]:
final_test_df.head()

,Provider,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,InscClaimAmtReimbursed,DeductibleAmtPaid,inpatient,AdmissionDt,DischargeDt,...,procedure_9,procedure_10,procedure_11,procedure_12,procedure_13,procedure_14,procedure_15,procedure_16,procedure_17,procedure_18
0,PRV51002,BENE13342,CLM129901,2009-01-10,2009-01-10,100,0.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,PRV51002,BENE13915,CLM163936,2009-01-29,2009-01-29,40,0.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,PRV51002,BENE13915,CLM348696,2009-05-09,2009-05-09,70,0.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,PRV51002,BENE13915,CLM500906,2009-08-01,2009-08-01,100,0.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,PRV51002,BENE14575,CLM272820,2009-03-29,2009-03-29,400,0.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [41]:
final_test_df.to_csv("processed_data/final_test_set.csv")